# Suffix Tree Term



#### Made by Yuliia Hetman

In [ ]:
alphabet="абвгґдеєжзиіїйклмнопрстуфхцчшщьюя"
ordnum = []
for letter in alphabet:
    ordnum.append(ord(letter))
    print(letter, ':', ord(letter))
print(len(alphabet))

alphabet="abcdefghijklmnoqprstuvwxyz"
print(len(alphabet))
print(sorted(ordnum))

In [ ]:
class TreeNode:
    def __init__(self):
        self.children = [None]*34
        # isEndOfWord is True if node represent the end of the word
        self.isEndOfWord = False
        
        

class Tree:
     
    # Trie data structure class
    def __init__(self):
        self.root = self.getNode()
 
    def getNode(self):
        # Returns new tree node (initialized to NULLs)
        return TreeNode()
 
    def _charToIndex(self,ch):
        # private helper function
        # Converts key current character into index
        # use only 'a' through 'z' and lower case
        i = 8217 if ord(ch) == 96 else ord(ch)
#         if i == 96:
#             i = 8217
        ukr = [1100, 1102, 1103, 1108, 1110, 1111, 1169, 8217] # 8217 - апостроф
        if i in ukr:
            return 1098 + ukr.index(i) - ord('а')
        return ord(ch) - ord('а')
 
 
    def insert(self,key):
         
        # If not present, inserts key into tree
        # If the key is prefix of tree node,
        # just marks leaf node
        pCrawl = self.root
        length = len(key)
        for level in range(length):
            index = self._charToIndex(key[level])
            
            # if current character is not present
            if not pCrawl.children[index]:
                pCrawl.children[index] = self.getNode()
            pCrawl = pCrawl.children[index]
 
        # mark last node as leaf
        pCrawl.isEndOfWord = True
 
    def search(self, key):
         
        # Search key in the trie
        # Returns true if key presents
        # in trie, else false
        pCrawl = self.root
        length = len(key)
        for level in range(length):
            index = self._charToIndex(key[level])
            if not pCrawl.children[index]:
                return False
            pCrawl = pCrawl.children[index]
        return pCrawl.isEndOfWord
 
    def haschild(self,node):
        for i in range(33):
            if node.children[i]:
                return True
        return False
      
    def displayUtil(self,visited,node,str):
        index = 0
        while index < 34:
            if node.children[index]:
                str += chr(1072 + index)
                #print(2,str)
                if node.children[index].isEndOfWord == False:
                    self.displayUtil(visited, node.children[index], str)
                    str = str[0 : (len(str) - 1)]
                else:
                    if str not in visited:
                        visited.append(str)
                    if self.haschild(node.children[index]):
                        self.displayUtil(visited, node.children[index], str)
                        str = str[0 : (len(str) - 1)]
                      
            index += 1
      
    def display(self):
        visited=[]
        str=''
        self.displayUtil(visited,self.root,str)
#         print("Content of Trie:")
#         for i in range(len(visited)):
#             print(visited[i])
        return visited

    def suggestionsRec(self, node, word):
 
        # Method to recursively traverse the trie
        # and return a whole word.
        if node.last:
            print(word)
        for a, n in node.children.items():
            self.suggestionsRec(n, word + a)
 
    def printAutoSuggestions(self, key):
 
        # Returns all the words in the trie whose common
        # prefix is the given key thus listing out all
        # the suggestions for autocomplete.
        node = self.root
        for a in key:
            # no string in the Trie has this prefix
            if not node.children.get(a):
                return 0
            node = node.children[a]
 
        # If prefix is present as a word, but
        # there is no subtree below the last
        # matching node.
        if not node.children:
            return -1
 
        self.suggestionsRec(node, key)
        return 1

In [ ]:
%%time
from multiprocessing import Pool
import re, os, json
from string import digits
import pandas as pd

docIDs = list(range(0, 18))

def delete_sings_digits(content):
    punc = '''!()-'”№[]{};:"\,“«»<>./?@#$%—…^&*_~|–abcdefghijklmnoqprstuvwxyz'''
    content = "".join([ele for ele in content.lower() if ele not in punc])
    table = str.maketrans('', '', digits)
    content = content.translate(table)
    return content


def create_vocabulary(content):    
    raw_content = delete_sings_digits(content)
    vocab = [i.lower() for i in raw_content.split() if i != '']
    return vocab

def read_book(book):
    dirname='../books/'
#     tree = Tree()
    with open(dirname + book, 'r', encoding='utf-8') as file:
        text = create_vocabulary(file.read())
#     unique = list(set(text))
#     for key in unique:
#         tree.insert(key)
    return text


def isUkrainian(word):
    ukrainian = [96, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1100, 1102, 1103, 1108, 1110, 1111, 1169, 8217]
    for l in word:
        if ord(l) not in ukrainian:
#             print("NOT UKRAINIAN LETTER:", l, "IN WORD", key)
            return False
    return True
       
        
def get_vocabulary(dirname='../books/'):
    books = os.listdir(dirname)
    vocabulary = list()
    dictionary = dict()
    memory = 0
    with Pool(8) as p:
        vocabul = p.map(read_book, books)
    for i, book in enumerate(books):
        dictionary[book.replace('.txt', '')] = vocabul[i]
        vocabulary += vocabul[i]
        print(book, "is read!")
        memory += os.stat(dirname + book).st_size
    return memory, vocabulary, dictionary


memory, vocabulary, dictionary = get_vocabulary()
print(f'Total number of words : {len(vocabulary)}')
print(f'Total number of tokens : {len(set(vocabulary))}')

class TreeNode():
    def __init__(self):
        # Initialising one node for trie
        self.children = {}
        self.last = False
 
 
class Tree():
    def __init__(self):
 
        # Initialising the trie structure.
        self.root = TrieNode()
 
    def formTree(self, keys):
 
        # Forms a trie structure with the given set of strings
        # if it does not exists already else it merges the key
        # into it by extending the structure as required
        for key in keys:
            self.insert(key)  # inserting one key to the trie.
 
    def insert(self, key):
 
        # Inserts a key into trie if it does not exist already.
        # And if the key is a prefix of the trie node, just
        # marks it as leaf node.
        node = self.root
 
        for a in key:
            if not node.children.get(a):
                node.children[a] = TreeNode()
 
            node = node.children[a]
 
        node.last = True
 
    def suggestionsRec(self, node, word):
 
        # Method to recursively traverse the trie
        # and return a whole word.
        if node.last:
            print(word)
 
        for a, n in node.children.items():
            self.suggestionsRec(n, word + a)
 
    def printAutoSuggestions(self, key):
 
        # Returns all the words in the trie whose common
        # prefix is the given key thus listing out all
        # the suggestions for autocomplete.
        node = self.root
 
        for a in key:
            # no string in the Trie has this prefix
            if not node.children.get(a):
                return 0
            node = node.children[a]
 
        # If prefix is present as a word, but
        # there is no subtree below the last
        # matching node.
        if not node.children:
            return -1
 
        self.suggestionsRec(node, key)
        return 1
    
tree = Tree()
ukr_voc = list()
for key in vocabulary:
    if isUkrainian(key) == True:
        ukr_voc.append(key)
        tree.insert(key)

In [ ]:
inverse = Tree()
for key in ukr_voc:
    k = key[::-1]
    inverse.insert(k)


In [ ]:
tree.printAutoSuggestions("шо")

In [ ]:
def find_jocker(string, tree, inverse):
    index_jocker = string.find("*")
    if index_jocker == -1 or index_jocker == len(string) - 1:
        return tree.printAutoSuggestions(string)
    start = string[:index_jocker]
    versionsStart = tree.printAutoSuggestions()

In [ ]:
inverse.printAutoSuggestions('кв')

In [ ]:
CHILDREN = 34
MAX = 100
 
# Trie node
class InverseTree:
     def __init__(self):
        # Initialising one node for trie
        self.children = {}
        self.last = False
 
    def __init__(self):
        self.child = [None for i in range(CHILDREN)]
         
        # endOfWord is true if the node represents
        # end of a word
        self.endOfWord = False

def createNode():
 
    temp = InverseTree()
    return temp
 
# Function will insert the string in a trie recursively
def insertRecursively(itr, str, i):
    if (i < len(str)):
        index = ord(str[i]) - ord('a')
        
        if (itr.children[index] == None):
            # Assigning a newly created node
            itr.children[index] = createNode()
         
        # Recursive call for insertion
        # of a string in a trie
        insertRecursively(itr.children[index], str, i + 1)
     
    else:
        # Make the endOfWord true which represents
        # the end of string
        itr.last = True

# Function call to insert a string
def insert(itr, str):
 
    # Function call with necessary arguments
    insertRecursively(itr, str, 0)

# Function to print traverse
# the tree in bottom to top manner
def printPattern(itr):
 
    # Base condition
    if(itr == None):
        return
 
    # Loop for printing t a value of character
    for i in range(CHILDREN):
 
        # Recursive call for print pattern
        printPattern(itr.children[i])
        if (itr.children[i] != None):
            ch = chr(i + 97)
             
            # Print character
            print(ch,end=', ')